In [1]:
%matplotlib inline
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import bnpy
import warnings
from scipy import stats
import random

from scipy.stats import kruskal, f_oneway
import collections
import scikit_posthocs as sp

warnings.filterwarnings('ignore')

In [2]:
pth = '../data/expression/target-high-risk-nbl-mycn-na-exp-2018-11-12.tsv'

exp = pd.read_csv(pth, sep='\t', index_col=0) 

In [3]:
pth = '../data/archive/mycn-na-nbl-0.2-assign-2019-03-02.tsv'

assign = pd.read_csv(pth, sep='\t', index_col=0)

In [4]:
exp.reindex(assign[assign['1'] == 1].index.values, axis=1).to_csv('target-high-risk-nbl-mycn-na-exp-2018-11-12-cluster0.tsv',
                                                                  sep='\t')

In [5]:
def n1(key, zscore):
    import uuid
    import subprocess
    import numpy as np
    
    _dir = '/tmp/%s' % str(uuid.uuid4())

    os.mkdir(_dir)

    cmd = ["Rscript",
           "/opt/hydra/bin/fgsea.R",
           "/opt/hydra/data/Human_GO_AllPathways_no_GO_iea_October_01_2018_symbol.gmt",
           os.path.join(_dir, "%s.rnk" % key),
           os.path.join(_dir, "%s.fgsea" % key)]

    zscore = zscore.sort_values(ascending=False)
    
    zscore.to_csv(os.path.join(_dir, "%s.rnk" % key),
                 header=None,
                 sep='\t')

    subprocess.check_call(cmd)
    
    return pd.read_csv(os.path.join(_dir, "%s.fgsea" % key), index_col=0)

In [6]:
RUN = False

if RUN: 
    _dir = '../data/gsea/'
    for cluster, rows in assign.groupby('1'):
        samps = rows.index.values
        back = [x for x in exp.columns if x not in samps]
    
        for samp in samps:
            zscore = (exp[samp] - exp[back].mean(axis=1)) / (exp[back].std(axis=1) + 0.05)
            res = n1(samp, zscore)
        
            pth = os.path.join(_dir, 'sub_cohort', str(cluster), samp)
            res.to_csv(pth)
        
            cback = [x for x in samps if x != samp]
            zscore = (exp[samp] - exp[cback].mean(axis=1)) / (exp[cback].std(axis=1) + 0.05)
            res2 = n1(samp, zscore)
        
            pth = os.path.join(_dir, 'sub_clust', str(cluster), samp)
            res2.to_csv(pth)
            
else:
    print("Run already")

In [13]:
_dir = '../data/gsea/'

sub_cohort = collections.defaultdict(list)
sub_cohort_unique = collections.defaultdict(list)
sub_clust_unique = collections.defaultdict(list)

sigs = set()

interest = ["REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1901342", 
            "POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017",
            "PLATELET DEGRANULATION%GOBP%GO:0002576",
            "RESPONSE TO ELEVATED PLATELET CYTOSOLIC CA2+%REACTOME DATABASE ID RELEASE 66%76005",
            "POSITIVE REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1904018"]

cluster1_metastatic = []
for cluster, rows in assign.groupby('1'):
    samps = rows.index.values
    for samp in samps:
        pth = os.path.join(_dir, 'sub_cohort', str(cluster), samp)        
        f1 = pd.read_csv(pth, index_col=0)
        f1 = f1[(f1['padj'] < 0.05) & (f1['ES'] > 0.0)]
        sigs.update(f1.index.values)
        
        pth = os.path.join(_dir, 'sub_clust', str(cluster), samp)
        f2 = pd.read_csv(pth, index_col=0)
        f2 = f2[(f2['padj'] < 0.05) & (f2['ES'] > 0.0)]
        sigs.update(f2.index.values)
        
        if cluster == 1:
            print samp
            print f1.sort_values('NES', ascending=False)['NES'].head(50)
            print '\n'
            print f2.sort_values('NES', ascending=False)['NES'].head(50)
            print '\n\n'
            
            if len(f2.index.intersection(interest)) > 0:
                cluster1_metastatic.append(samp)
        
        sub_cohort[cluster].extend(list(f1.index.values))
        sub_cohort_unique[cluster].extend(list(set(f1.index.values) - set(f2.index.values)))
        sub_clust_unique[cluster].extend(list(set(f2.index.values) - set(f1.index.values)))

TARGET-30-PALETP-01
pathway
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958                                                                     4.806437
COMPLEMENT ACTIVATION%GOBP%GO:0006956                                                                                        4.791185
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455                                               4.761391
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814                                                                                       4.750599
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                            4.646468
IMMUNOGLOBULIN COMPLEX, CIRCULATING%GOCC%GO:0042571                                                                          4.601453
IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987                                                                              4.433994
IMMUNOGLOBULIN MEDIATED IMMUNE RES

TARGET-30-PALTEG-01
pathway
EUKARYOTIC TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%156842                                                                        7.203133
PEPTIDE CHAIN ELONGATION%REACTOME DATABASE ID RELEASE 66%156902                                                                                 7.116896
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689                                                                  7.110434
VIRAL MRNA TRANSLATION%REACTOME%R-HSA-192823.3                                                                                                  6.957324
EUKARYOTIC TRANSLATION TERMINATION%REACTOME%R-HSA-72764.4                                                                                       6.924007
L13A-MEDIATED TRANSLATIONAL SILENCING OF CERULOPLASMIN EXPRESSION%REACTOME DATABASE ID RELEASE 66%156827                                        6.785744
NONSENSE MEDIATED DECAY (NMD) INDEPENDENT OF THE EXON 

TARGET-30-PANRHJ-01
Series([], Name: NES, dtype: float64)


Series([], Name: NES, dtype: float64)



TARGET-30-PANZVU-01
pathway
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                               2.967901
ANTIGEN RECEPTOR-MEDIATED SIGNALING PATHWAY%GOBP%GO:0050851                                                                     2.737390
ANTIGEN ACTIVATES B CELL RECEPTOR (BCR) LEADING TO GENERATION OF SECOND MESSENGERS%REACTOME DATABASE ID RELEASE 66%983695       2.637462
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814                                                                                          2.585788
B CELL ACTIVATION%GOBP%GO:0042113                                                                                               2.576861
PID_TCR_PATHWAY%MSIGDB_C2%PID_TCR_PATHWAY                                                                                       2.528281
TCR SIGNALING IN NAIVE CD4+ T CELLS%PATHWAY INTER

TARGET-30-PAPKXS-01
pathway
RUNX1 REGULATES GENES INVOLVED IN MEGAKARYOCYTE DIFFERENTIATION AND PLATELET FUNCTION%REACTOME DATABASE ID RELEASE 66%8936459              2.569351
DNA METHYLATION%REACTOME DATABASE ID RELEASE 66%5334118                                                                                    2.516333
MEIOTIC SYNAPSIS%REACTOME%R-HSA-1221632.1                                                                                                  2.456418
PRC2 METHYLATES HISTONES AND DNA%REACTOME%R-HSA-212300.1                                                                                   2.432881
MEIOSIS%REACTOME DATABASE ID RELEASE 66%1500620                                                                                            2.417536
PACKAGING OF TELOMERE ENDS%REACTOME DATABASE ID RELEASE 66%171306                                                                          2.359921
MEIOTIC RECOMBINATION%REACTOME DATABASE ID RELEASE 66%912446                        

TARGET-30-PAPUEB-01
pathway
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                                            3.739674
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814                                                                                                       3.571827
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958                                                                                     3.568510
IMMUNOGLOBULIN COMPLEX, CIRCULATING%GOCC%GO:0042571                                                                                          3.524145
IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987                                                                                              3.514478
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455                                                               3.463145
B CELL MEDIATED IMMUNITY%GOBP%GO:0019724                                

...

TARGET-30-PASRFS-01
pathway
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                                            3.737129
HALLMARK_INTERFERON_ALPHA_RESPONSE%MSIGDB_C2%HALLMARK_INTERFERON_ALPHA_RESPONSE                                                              3.536853
ANTIGEN RECEPTOR-MEDIATED SIGNALING PATHWAY%GOBP%GO:0050851                                                                                  3.466021
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958                                                                                     3.433202
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455                                                               3.407572
HALLMARK_INTERFERON_GAMMA_RESPONSE%MSIGDB_C2%HALLMARK_INTERFERON_GAMMA_RESPONSE                                                              3.386763
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814                                  

TARGET-30-PASWIJ-01
pathway
LYMPHOCYTE ACTIVATION%GOBP%GO:0046649                                                                                           3.668840
IMMUNOREGULATORY INTERACTIONS BETWEEN A LYMPHOID AND A NON-LYMPHOID CELL%REACTOME DATABASE ID RELEASE 66%198933                 3.536392
T CELL ACTIVATION%GOBP%GO:0042110                                                                                               3.532621
HALLMARK_ALLOGRAFT_REJECTION%MSIGDB_C2%HALLMARK_ALLOGRAFT_REJECTION                                                             3.342802
LYMPHOCYTE DIFFERENTIATION%GOBP%GO:0030098                                                                                      3.238714
REGULATION OF LYMPHOCYTE ACTIVATION%GOBP%GO:0051249                                                                             3.221555
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                               3.204417
ANTIGEN RECEP

TARGET-30-PASXRJ-01
pathway
ANTIGEN RECEPTOR-MEDIATED SIGNALING PATHWAY%GOBP%GO:0050851                                                                                  3.578097
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                                            3.537792
ANTIGEN BINDING%GOMF%GO:0003823                                                                                                              3.320441
LYMPHOCYTE ACTIVATION%GOBP%GO:0046649                                                                                                        3.302679
ADAPTIVE IMMUNE RESPONSE%GOBP%GO:0002250                                                                                                     3.264245
REGULATION OF MONONUCLEAR CELL PROLIFERATION%GOBP%GO:0032944                                                                                 3.234444
POSITIVE REGULATION OF LYMPHOCYTE ACTIVATION%GOBP%GO:0051251            

TARGET-30-PATDXC-01
pathway
HALLMARK_INTERFERON_ALPHA_RESPONSE%MSIGDB_C2%HALLMARK_INTERFERON_ALPHA_RESPONSE                                                              4.089529
IMMUNOREGULATORY INTERACTIONS BETWEEN A LYMPHOID AND A NON-LYMPHOID CELL%REACTOME DATABASE ID RELEASE 66%198933                              3.881095
RESPONSE TO TYPE I INTERFERON%GOBP%GO:0034340                                                                                                3.858775
REGULATION OF LYMPHOCYTE ACTIVATION%GOBP%GO:0051249                                                                                          3.844962
DEFENSE RESPONSE TO OTHER ORGANISM%GOBP%GO:0098542                                                                                           3.819389
HALLMARK_INTERFERON_GAMMA_RESPONSE%MSIGDB_C2%HALLMARK_INTERFERON_GAMMA_RESPONSE                                                              3.812222
POSITIVE REGULATION OF LYMPHOCYTE ACTIVATION%GOBP%GO:0051251            

TARGET-30-PATINJ-01
pathway
HALLMARK_E2F_TARGETS%MSIGDB_C2%HALLMARK_E2F_TARGETS                                                     3.014026
U2-TYPE SPLICEOSOMAL COMPLEX%GOCC%GO:0005684                                                            2.767973
DNA REPLICATION%REACTOME DATABASE ID RELEASE 66%69306                                                   2.767758
MITOCHONDRIAL TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%5389840                            2.724842
DNA REPLICATION PRE-INITIATION%REACTOME DATABASE ID RELEASE 66%69002                                    2.706360
MITOCHONDRIAL TRANSLATIONAL ELONGATION%GOBP%GO:0070125                                                  2.692239
MITOCHONDRIAL TRANSLATION TERMINATION%REACTOME DATABASE ID RELEASE 66%5419276                           2.684362
SPLICEOSOMAL SNRNP COMPLEX%GOCC%GO:0097525                                                              2.681815
MITOCHONDRIAL TRANSLATIONAL TERMINATION%GOBP%GO:0070126             

In [14]:
len(sigs)

10720

In [15]:
for cluster, counts in sub_cohort_unique.items():
    print cluster, len(assign[assign['1'] == cluster])
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(50):
        print gs, n
    
    print '\n\n'

1 31
POSITIVE T CELL SELECTION%GOBP%GO:0043368 23
THYMIC T CELL SELECTION%GOBP%GO:0045061 23
ALPHA-BETA T CELL RECEPTOR COMPLEX%GOCC%GO:0042105 22
POSITIVE THYMIC T CELL SELECTION%GOBP%GO:0045059 22
T CELL RECEPTOR COMPLEX%GOCC%GO:0042101 21
T CELL DIFFERENTIATION%GOBP%GO:0030217 20
T CELL SELECTION%GOBP%GO:0045058 19
ESTABLISHMENT OF T CELL POLARITY%GOBP%GO:0001768 19
REGULATION OF DENDRITIC CELL DENDRITE ASSEMBLY%GOBP%GO:2000547 19
BIOCARTA_TCRA_PATHWAY%MSIGDB_C2%BIOCARTA_TCRA_PATHWAY 18
BIOCARTA_TCYTOTOXIC_PATHWAY%MSIGDB_C2%BIOCARTA_TCYTOTOXIC_PATHWAY 18
BIOCARTA_THELPER_PATHWAY%MSIGDB_C2%BIOCARTA_THELPER_PATHWAY 18
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853 17
T CELL DIFFERENTIATION IN THYMUS%GOBP%GO:0033077 16
ANTIGEN PROCESSING AND PRESENTATION, EXOGENOUS LIPID ANTIGEN VIA MHC CLASS IB%GOBP%GO:0048007 16
ENDOGENOUS LIPID ANTIGEN BINDING%GOMF%GO:0030883 16
ANTIGEN PROCESSING AND PRESENTATION, ENDOGENOUS LIPID ANTIGEN VIA MHC CLASS IB%GOBP%GO:0048006 16
IMMUNOGLOBULIN COMPLE

In [16]:
met = ['POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017',
       'POSITIVE REGULATION OF CELL MIGRATION%GOBP%GO:0030335',
       'POSITIVE REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1904018',
       'REGULATION OF ENDOTHELIAL CELL MIGRATION%GOBP%GO:0010594',
       'POSITIVE REGULATION OF CELLULAR COMPONENT MOVEMENT%GOBP%GO:0051272',
       'POSITIVE REGULATION OF CELL MOTILITY%GOBP%GO:2000147',
       'FOCAL ADHESION%GOCC%GO:0005925']

In [18]:
top5 = []

gs_plot = pd.DataFrame(columns=['cluster', 'gs', 'percent'])
for cluster, counts in sub_clust_unique.items():
    N = len(assign[assign['1'] == cluster])
    
    print cluster, N
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(10):
        print gs, n
        if cluster == 1:
            top5.append(gs)
        
    print '\n\n'

1 31
ADHERENS JUNCTION%GOCC%GO:0005912 10
ANCHORING JUNCTION%GOCC%GO:0070161 10
CELL-SUBSTRATE JUNCTION%GOCC%GO:0030055 9
CELL-SUBSTRATE ADHERENS JUNCTION%GOCC%GO:0005924 8
EXTRACELLULAR MATRIX%GOCC%GO:0031012 8
REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1901342 8
FOCAL ADHESION%GOCC%GO:0005925 8
POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017 8
POSITIVE REGULATION OF CELL MIGRATION%GOBP%GO:0030335 8
POSITIVE REGULATION OF CELL MOTILITY%GOBP%GO:2000147 8



2 26
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455 8
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958 8
IMMUNOGLOBULIN COMPLEX%GOCC%GO:0019814 8
LEUKOCYTE DEGRANULATION%GOBP%GO:0043299 8
MYELOID LEUKOCYTE MEDIATED IMMUNITY%GOBP%GO:0002444 8
IMMUNOGLOBULIN RECEPTOR BINDING%GOMF%GO:0034987 7
GRANULOCYTE ACTIVATION%GOBP%GO:0036230 7
INFLAMMATORY RESPONSE%GOBP%GO:0006954 7
CELL ACTIVATION INVOLVED IN IMMUNE RESPONSE%GOBP%GO:0002263 7
NEUTROPHIL ACTIVATION%GOBP%GO:0042119 7



3 13
CONDENSED 

In [19]:
top5

['ADHERENS JUNCTION%GOCC%GO:0005912',
 'ANCHORING JUNCTION%GOCC%GO:0070161',
 'CELL-SUBSTRATE JUNCTION%GOCC%GO:0030055',
 'CELL-SUBSTRATE ADHERENS JUNCTION%GOCC%GO:0005924',
 'EXTRACELLULAR MATRIX%GOCC%GO:0031012',
 'REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1901342',
 'FOCAL ADHESION%GOCC%GO:0005925',
 'POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017',
 'POSITIVE REGULATION OF CELL MIGRATION%GOBP%GO:0030335',
 'POSITIVE REGULATION OF CELL MOTILITY%GOBP%GO:2000147']

In [20]:
def get_event(event):
    if pd.isnull(event):
        print("NULL")
        return np.nan
    
    events = ['Relapse',
              'Death', 
              'Progression',
              'Event',
              'Second Malignant Neoplasm']
    
    if event == 'Censored':
        return 0
    
    elif event in events:
        return 1 
    
    else:
        raise ValueError(event)
        
def get_vital(vital):
    if pd.isnull(vital):
        print("NULL")
        return np.nan
    
    
    if vital == 'Alive':
        return 0
    
    elif vital == 'Dead':
        return 1
    
    else:
        raise ValueError(vital)

pth = '../data/meta/TARGET_NBL_ClinicalData_Discovery_20170525.xlsx'
clinical = pd.read_excel(pth, index_col=0)

surv = pd.DataFrame(index=assign[assign['1'] == 2].index, 
                    columns=['OS', 'vital', 'EFS', 'event', 'immune'])

for sample in assign[assign['1'] == 1].index:
    root = '-'.join(sample.split('-')[:3])
    if root not in clinical.index:
        print "Missing: ", sample
        continue
    
    OS = clinical.loc[root, 'Overall Survival Time in Days'].item()
    vital = get_vital(clinical.loc[root, 'Vital Status'])
    
    EFS = clinical.loc[root, 'Event Free Survival Time in Days'].item()
    event = get_event(clinical.loc[root, 'First Event'])
    
    
    immune = 1 if sample in cluster1_metastatic else 0
    surv.loc[sample, :] = [OS, vital, EFS, event, immune]

In [21]:
import uuid

# Can't figure out how to pass this dataframe
survpth = '../data/mycn-na-tme-signal-subtract-cluster1-survival.tsv'
surv.to_csv(survpth, sep='\t')

%%R -i survpth

require(dpylr)
require(survival)
require(survminer)

surv <- read.table(survpth,
                  sep='\t',
                  header=T)

surv <- dplyr::as_tibble(surv)

%%R

fit <- survfit(Surv(OS, vital) ~ immune, data=surv)
ggsurvplot(fit, conf.int=F, pval=T, risk.table=TRUE)

%%R

fit <- survfit(Surv(EFS, event) ~ immune, data=surv)
ggsurvplot(fit, conf.int=F, pval=T, risk.table=TRUE)